### Experiments and Approaches - SRNet (Steganalysis Residual Network) #2

This experiment is an extended version of SRNet #1 - here, we tried to train the model on a balanced dataset of 200+200 cover and stego images. We kept the model running for 18 hours.

In [1]:
import numpy as np
from PIL import Image
import os

import keras 
from keras import Model
from keras.layers import Input, Dense, ReLU ,Conv2D, Flatten , Add, Softmax , AveragePooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.losses import binary_crossentropy , categorical_crossentropy

Using TensorFlow backend.


In [2]:
def layer_type1(inputs,filter_size):
    x = Conv2D(kernel_size=(3,3),
               filters= filter_size,
               padding="same",input_shape = (256,256))(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x

def layer_type2(inputs,filter_size=16):
    x = layer_type1(inputs,filter_size=filter_size)
    
    x = Conv2D(kernel_size=(3,3),
               filters= filter_size,
               padding="same")(x)
    
    x = BatchNormalization()(x)
    
    out = Add()([x, inputs])
    
    return out

def layer_type3(inputs,filter_size):
    
    x = layer_type1(inputs,filter_size=filter_size)
    
    x = Conv2D(kernel_size=(3,3),
               filters= filter_size,
               padding="same")(x)
    
    x = BatchNormalization()(x)
    
    x = AveragePooling2D(pool_size=(2,2),strides=2)(x)
    
    y = Conv2D(kernel_size=(1,1),
               filters= filter_size,
               padding="same",strides=2)(inputs)
    y = BatchNormalization()(y)
    
    out = Add()([x, y])
    
    return out

def layer_type4(inputs,filter_size=512):
    x = layer_type1(inputs,filter_size=filter_size)
    x = Conv2D(kernel_size=(16,16),
               filters= filter_size,
               padding="same")(x)
    x = BatchNormalization()(x)
    x = GlobalAveragePooling2D()(x)
    return x

In [3]:
def create_SRNet():
    init_input = Input(shape=(256,256,1))
    
    L1 = layer_type1(init_input,filter_size=64)
    L2 = layer_type1(L1,filter_size=16)
    
    L3 = layer_type2(L2)
    L4 = layer_type2(L3)
    L5 = layer_type2(L4)
    L6 = layer_type2(L5)
    L7 = layer_type2(L6)
    
    L8 = layer_type3(L7,filter_size=16)
    L9 = layer_type3(L8,filter_size=64)
    L10 = layer_type3(L9,filter_size=128)
    L11 = layer_type3(L10,filter_size=256)
    
    L12 = layer_type4(L11,filter_size=512)
    
    #L13 = Flatten()(L12)
    L14 = Dense(2)(L12)
    output = Softmax()(L14)
    
    model = Model(init_input,output,)
    model._name = "SRNet"
    model.compile(optimizer = Adam(lr=0.001) , 
               loss = "binary_crossentropy" , metrics = ['accuracy'])
    
    return model

In [4]:
test = create_SRNet()
test.summary()

Model: "SRNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 64) 256         conv2d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 256, 256, 64) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [5]:
train = os.listdir('/home/iamsmit96/bossdata/final_dataset/train/cover')
validation = os.listdir('/home/iamsmit96/bossdata/final_dataset/validation/cover')

['1179.pgm',
 '118.pgm',
 '1180.pgm',
 '1181.pgm',
 '1182.pgm',
 '1183.pgm',
 '1184.pgm',
 '1185.pgm',
 '1186.pgm',
 '1187.pgm',
 '1188.pgm',
 '1189.pgm',
 '119.pgm',
 '1190.pgm',
 '1191.pgm',
 '1192.pgm',
 '1193.pgm',
 '1194.pgm',
 '1195.pgm',
 '1196.pgm',
 '1197.pgm',
 '1198.pgm',
 '1199.pgm',
 '12.pgm',
 '120.pgm',
 '1200.pgm',
 '1201.pgm',
 '1202.pgm',
 '1203.pgm',
 '1204.pgm',
 '1205.pgm',
 '1206.pgm',
 '1207.pgm',
 '1208.pgm',
 '1209.pgm',
 '121.pgm',
 '1210.pgm',
 '1211.pgm',
 '1212.pgm',
 '1213.pgm',
 '1214.pgm',
 '1215.pgm',
 '1216.pgm',
 '1217.pgm',
 '1218.pgm',
 '1219.pgm',
 '122.pgm',
 '1220.pgm',
 '1221.pgm',
 '1222.pgm',
 '1223.pgm',
 '1224.pgm',
 '1225.pgm',
 '1226.pgm',
 '1227.pgm',
 '1228.pgm',
 '1229.pgm',
 '123.pgm',
 '1230.pgm',
 '1231.pgm',
 '1232.pgm',
 '1233.pgm',
 '1234.pgm',
 '1235.pgm',
 '1236.pgm',
 '1237.pgm',
 '1238.pgm',
 '1239.pgm',
 '124.pgm',
 '1240.pgm',
 '1241.pgm',
 '1242.pgm',
 '1243.pgm',
 '1244.pgm',
 '1245.pgm',
 '1246.pgm',
 '1247.pgm',
 '1248.p

In [7]:
def get_data(typee,listt):
    image_data =[]
    label=[]
    if listt == train:
        num = 200
    else:
        num = 100
    for i in range(0,num):
        image1 = np.asarray(Image.open('/home/iamsmit96/bossdata/final_dataset/{}/cover/{}'.format(typee,listt[i])))  
        image2 = np.asarray(Image.open('/home/iamsmit96/bossdata/final_dataset/{}/stego/{}'.format(typee,listt[i])))
        image_data.append([image1,0])
        image_data.append([image2,1])
    return image_data

In [8]:
train_set = get_data('train',train)

In [9]:
valid_set = get_data('validation',validation)

In [10]:
len(valid_set) , len(train_set)

(200, 400)

In [12]:
x_train = []
y_train = []
x_val = []
y_val = []

In [14]:
for feature, label in train_set:
        x_train.append(feature)
        y_train.append(label)

In [13]:
for feature, label in valid_set:
        x_val.append(feature)
        y_val.append(label)

In [ ]:
%%time
test.fit(np.array(x_train),np.array(y_train),batch_size=20,validation_data=(np.array(x_val),np.array(y_val)),
         epochs=50,use_multiprocessing=True, workers=8)

Epoch 1/50
20/20 [==============================] - 3242s 162s/step - loss: 1.1275 - accuracy: 0.4850 - val_loss: 7.4795 - val_accuracy: 0.5000
Epoch 2/50
20/20 [==============================] - 3249s 162s/step - loss: 0.7040 - accuracy: 0.4500 - val_loss: 7.2946 - val_accuracy: 0.5000
Epoch 3/50
20/20 [==============================] - 3224s 161s/step - loss: 0.7088 - accuracy: 0.5525 - val_loss: 2.5149 - val_accuracy: 0.5000
Epoch 4/50
20/20 [==============================] - 3211s 161s/step - loss: 0.7036 - accuracy: 0.4975 - val_loss: 0.9464 - val_accuracy: 0.5000
Epoch 5/50
20/20 [==============================] - 3209s 160s/step - loss: 0.6961 - accuracy: 0.4675 - val_loss: 0.8263 - val_accuracy: 0.5000
Epoch 6/50
20/20 [==============================] - 3217s 161s/step - loss: 0.6952 - accuracy: 0.4925 - val_loss: 0.8097 - val_accuracy: 0.5000
Epoch 7/50
20/20 [==============================] - 3240s 162s/step - loss: 0.6961 - accuracy: 0.4900 - val_loss: 0.7692 - val_accuracy:

In [ ]:
This training session was running parallel